In [1]:
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset
import soundfile as sf

/private/l2hu/interqu/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")



In [3]:
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)


Extracting data files: 100%|████████| 1/1 [00:00<00:00,  2.69it/s]


Dataset librispeech_asr_dummy downloaded and prepared to /home/l2hu/.cache/huggingface/datasets/patrickvonplaten___librispeech_asr_dummy/clean/2.1.0/f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc. Subsequent calls will reuse this data.


In [14]:

inputs = processor(ds["speech"][0], sampling_rate=16_000, return_tensors="pt")
# generated_ids = model.generate(input_ids=inputs["input_features"], attention_mask=inputs["attention_mask"])
generated_ids = model.generate(input_features=inputs["input_features"], attention_mask=inputs["attention_mask"])

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [13]:
transcription = processor.batch_decode(generated_ids)

In [15]:
transcription

['</s> mister quilter is the apostle of the middle classes and we are glad to welcome his gospel</s>']

In [16]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using {torch.cuda.get_device_name()} GPU for inference.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU for inference.")

Using NVIDIA GeForce GTX 1070 GPU for inference.


In [29]:
inputs = processor(ds["speech"][2], sampling_rate=16_000, return_tensors="pt").to(device)
# generated_ids = model.generate(input_ids=inputs["input_features"], attention_mask=inputs["attention_mask"])
generated_ids = model.generate(input_features=inputs["input_features"].to(device), attention_mask=inputs["attention_mask"].to(device))

In [27]:
transcription = processor.batch_decode(generated_ids)

In [28]:
transcription

['</s> he tells us that at this festive season of the year with christmas and roast beef looming before us similes drawn from eating and its results occur most readily to the mind</s>']

In [26]:
len(ds["speech"])

73